In [1]:
import paho.mqtt.client as mqtt
import sys
from gtts import gTTS
import os
import threading as threading, time
from threading import Thread

In [2]:
#Callback - conexao ao broker realizada
def on_connect(client, userdata, flags, rc):
    print(f'[STATUS] Conectado ao Broker. Resultado de conexao: {str(rc)}')
 
    #faz subscribe automatico no topico
    client.subscribe('pythonTodoDia')

In [3]:
#Callback - mensagem recebida do broker
def on_message(client, userdata, msg):
    mensagem_recebida = str(msg.payload)
    
    print(f'[MSG RECEBIDA] Topico: {msg.topic} Mensagem: {mensagem_recebida}')
    converter_texto_voz(mensagem_recebida)
    return mensagem_recebida

In [4]:
#callback - converter texto em voz
def converter_texto_voz(mensagem_recebida):
    #rotina de conversao texto em voz
    mensagem = str(mensagem_recebida[1:])
    print(mensagem)
    audio_texto = gTTS(text=mensagem, lang='pt')
    audio_texto.save('audio.mp3')

In [5]:
class Audio_thread(threading.Thread):
    def __init__(self, meuId, cont, mutex):
        self.meuId = meuId
        self.cont = cont
        self.mutex = mutex
        threading.Thread.__init__(self)
    
    #callback - reproduzir audio
    def run(self):
        while(True):
            with self.mutex:
                os.system('audio.mp3')
                time.sleep(5)

In [ ]:
stdoutmutex = threading.Lock()
threads = []
thread1 = Audio_thread(1, 100, stdoutmutex)
thread1.daemon = True
thread1.start()

for thread in threads:
    thread.join()

#programa principal:
try:
    print('[STATUS] Inicializando MQTT...')
    #inicializa MQTT:
    client = mqtt.Client()

    client.on_connect = on_connect
    client.on_message = on_message  

    client.connect('mqtt.eclipse.org', 1883, 80)
   
    client.loop_forever() 
    print('Saindo da thread principal')

except KeyboardInterrupt:
    print ('\nCtrl+C pressionado, encerrando aplicacao e saindo...')
    sys.exit(0)

[STATUS] Inicializando MQTT...
[STATUS] Conectado ao Broker. Resultado de conexao: 0
[MSG RECEBIDA] Topico: pythonTodoDia Mensagem: b'um dia eu compro uma v-lux'
'um dia eu compro uma v-lux'
[MSG RECEBIDA] Topico: pythonTodoDia Mensagem: b'que tal andar bicicleta'
'que tal andar bicicleta'
[MSG RECEBIDA] Topico: pythonTodoDia Mensagem: b'que tal andar de bicicleta'
'que tal andar de bicicleta'
